# Definición, tipologías y casos de uso de Graph Neural Networks para el aprendizaje basado en relaciones: presentación de implementación

## Introducción al notebook

Este documento de tipo notebook acompaña a la memoria del TFG con título *Definición, tipologías y casos de uso de Graph Neural Networks para el aprendizaje basado en relaciones*. El objetivo del documento es presentar la implementación de las pruebas realizadas con diferentes modelos de redes neuronales gráficas explicadas en el proyecto, así como plantear distintas aproximaciones para su codificación.

Aunque van a explicarse algunos conceptos relacionados con las librerías de Python utilizadas, se deja en manos de la memoria de proyecto el explicar la base teórica de los mismos, así como la evaluación e interpretación de los resultados obtenidos.

## Estructura del documento

Mediante el uso del orden presentado en la memoria, se van a repasar distintas aproximaciones a las GNN con dos planteamientos principales:

1. Clasificación de nodos dentro de un grafo:
    1. Implementación mediante un modelo con base espectral.
    1. Implementación con un modelo espacial mediante paso de mensajes.
    1. Contraste de resultados frente a un modelo tradicional que no tiene en cuenta la estructura de los datos, en busca de justificar la motivación de la existencia de las redes gráficas.
1. Predicción de enlazado.
    1. Implementación de un modelo que hace uso de paso de mensajes para predecir enlaces entre nodos.

Para cada uno de estos ejemplos se presentarán resultados de métricas de precisión durante el proceso de entrenamiento, para lo que se hará uso de datos de validación, así como pruebas finales sobre datos de test.

## Preparación del entorno

### Instalación de módulos

Como primer paso, se procede a instalar los módulos necesarios:

* [numpy](https://numpy.org/), para poder realizar las operaciones necesarias sobre los datos.
* [TensorFlow](https://www.tensorflow.org/), como librería principal para ejecutar los algoritmos de aprendizaje computacional del bloque de clasificación de nodos.
* [Spektral](https://graphneural.network/), una cómoda librería y colección de sets de datos que facilita el trabajo con redes neuronales gráficas.
* [PyTorch](https://pytorch.org/), librería de aprendizaje computacional que será usada para las pruebas de predicción de enlaces. Junto a ella se instalarán también algunas sublibrerías que serán necesarias.
* [tqdm](https://github.com/tqdm/tqdm), una librería simple para crear barras de progreso, con el simple objetivo de facilitar la lectura de algunas pruebas.

A continuación, se hacen las llamadas oportunas para instalar las mismas en el entorno actual. De manera adicional, se adjunta el fichero `requirements.txt` junto con este notebook para su uso en caso de necesidad.

In [2]:
# Install required modules
necessary_modules = ['numpy', 'tensorflow', 'spektral', 'torch', 'torch-cluster', 'torch-scatter', 'torch-sparse', 'torch-geometric', 'tqdm']
for module in necessary_modules:
    !pip install {module}

# Import numpy module, that is going to be use across the project
import numpy as np

### Configuración de TensorBoard

Tanto en los ejemplos donde se hace uso de TensorFlow como en aquellos implementados con PyTorch, se extraerán las métricas de aprendizaje y precisión a ficheros de registro que podrán ser leídos mediante [TensorBoard](https://www.tensorflow.org/tensorboard?hl=en), un entorno del propio TensorFlow que facilita su seguimiento y visualización. 

Para ello, es preciso preparar un directorio donde guardar el historial de registro de mensajes y configurar el propio entorno.

In [7]:
# Import TensorBoard callback to be able to use it in all the code samples
from keras.callbacks import TensorBoard

# Prepare placement for the logs
import os
root_logdir = os.path.join(os.curdir, 'my_logs')

# Create directory if it doesn't exists
from pathlib import Path
Path(root_logdir).mkdir(parents=True, exist_ok=True)

# Define a function that returns the path to store the log depending on the experiment
def get_run_logdir(experiment_name):
    import time
    run_id = time.strftime(f'run_{experiment_name}_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

## Implementación de pruebas

A continuación se presentan las distintas pruebas realizadas.

### Pruebas de clasificación de nodos

Tal como se expone en la memoria de proyecto, las redes neuronales gráficas permiten, entre otras cosas, la clasificación de nodos en base a la información de sus vecindarios, lo que permite asignar clases dentro del contexto de las entidades de un grafo.